# 使用asyncio包处理并发

## 18.1 线程与协程对比

[spinner_thread.py](./spinner_thread.py)

In [1]:
'\b' # '\x08' 退格符

'\x08'

## python3.5 后语法发生变化，书中的代码已经不再适用，故使用官网的代码

# 协程与任务

## 协程

In [2]:
import asyncio

async def main():
    print('Hello ...')
    await asyncio.sleep(1)
    print('... World!')

# asyncio.run(main()) 
# cause RuntimeError: This event loop is already running

# this is because the loop is already running in jupyter, so we can just use
# await main() to run the coroutine
await main()

Hello ...
... World!


In [15]:
asyncio.get_event_loop()

<_WindowsSelectorEventLoop running=True closed=False debug=False>

In [3]:
main()

<coroutine object main at 0x00000256ECD31A10>

In [4]:
import asyncio
import time

async def say_after(delay, what):
    await asyncio.sleep(delay)
    print(what)

async def main():
    print(f"started at {time.strftime('%X')}")
    await say_after(1, 'hello')
    await say_after(2, 'world')

    print(f"finished at {time.strftime('%X')}")

await main()

started at 16:53:54
hello
world
finished at 16:53:58


In [6]:
async def main():
    task1 = asyncio.create_task(
        say_after(1, 'hello'))
    task2 = asyncio.create_task(
        say_after(2, 'world'))
    
    print(f"started at {time.strftime('%X')}")

    # Wait until both tasks are completed (should take
    # around 2 seconds.)

    await task1
    await task2

    print(f"finished at {time.strftime('%X')}")

await main()

started at 16:56:29
hello
world
finished at 16:56:31


## 可等待对象

*可等待*对象有三种主要类型：**协程**、**任务**和**Future**。

In [12]:
# 协程 （coroutine)
import asyncio

async def nested():
    return 42

async def main():
    # Nothing happens if we just call "nested()".
    # A coroutine object is created but not awaited,
    # so it *won't run at all*.
    # nested()

    # Let's do it differently now and await it:
    print(await nested())  # will print "42".

await main()

42


In [13]:
# 任务 （Task)
import asyncio

async def nested():
    return 42

async def main():
    # Schedule nested() to run soon concurrently
    # with "main()".
    task = asyncio.create_task(nested())

    # "task" can now be used to cancel "nested()", or
    # can simply be awaited to wait until it is complete:
    print(await task)


await main()

42


```python
# Futures
# Future 是一种特殊的低层级可等待对象，表示一个异步操作的【最终结果】。
async def main():
    await function_that_returns_a_future_object()

    # this is also valid:
    await asyncio.gather(
        function_that_returns_a_future_object(),
        some_python_coroutine()
    )
```

## 运行asyncio程序

```python
async def main():
    await function()
    print('hello')

asyncio.run(main())
```

## 创建任务

```python
background_task = set()

for i in range(10):
    task = asyncio.create_task(some_coro(param=i))

    # Add task to the set. This creates a strong reference

    # To prevent keeping references to finished tasks forever
    # make each task remove its own reference from the set after
    # completion using the `add_done_callback()` method.
    task.add_done_callback(background_task.remove)
```

## 休眠

In [18]:
import asyncio
import datetime

async def display_data():
    loop = asyncio.get_running_loop()
    end_time = loop.time() + 5.0
    while True:
        print(datetime.datetime.now())
        if (loop.time() + 1.0) >= end_time:
            break
        await asyncio.sleep(1)

# asyncio.run(display_data())
await display_data()
    

2023-07-25 18:53:59.767432
2023-07-25 18:54:00.772755
2023-07-25 18:54:01.781676
2023-07-25 18:54:02.790837
2023-07-25 18:54:03.793948


## 并发运行任务

In [19]:
import asyncio

async def factorial(name, number):
    f = 1
    for i in range(2, number+1):
        print(f"Task {name}: Compute factorial({i})...")
        await asyncio.sleep(1)
        f *= i
    print(f"Task {name}: factorial({i}) = {f}")
    return f

async def main():
    # Schedule three calls *concurrently*:
    L = await asyncio.gather(
        factorial("A", 2),
        factorial("B", 3),
        factorial("C", 4),
    )
    print(L)

await main()

Task A: Compute factorial(2)...
Task B: Compute factorial(2)...
Task C: Compute factorial(2)...
Task A: factorial(2) = 2
Task B: Compute factorial(3)...
Task C: Compute factorial(3)...
Task B: factorial(3) = 6
Task C: Compute factorial(4)...
Task C: factorial(4) = 24
[2, 6, 24]


## 超时

In [1]:
import asyncio

async def eternity():
    # Sleep for one hour
    await asyncio.sleep(3600)
    print('yay!')

async def main():
    # Wait for at most 1 second
    try:
        await asyncio.wait_for(eternity(), timeout=1.0)
    except asyncio.TimeoutError:
        print('timeout!')

await main()

timeout!


## 简单等待

In [2]:
async def foo():
    return 42

task = asyncio.create_task(foo())
done, pending = await asyncio.wait({task})

if task in done:
    print('done')

done


```python
for coro in asyncio.as_completed(coro_list):
    result = await coro
    # process result
```